In [1]:
import time
from klampt import *
from klampt.math import vectorops,so3,se3
from IPython.display import clear_output
import ipywidgets as widgets
import ipyklampt
import math

world = WorldModel()
res = world.loadElement("data/robots/3R_zyy.rob")
kvis = ipyklampt.KlamptWidget(world,width=600,height=400)

robot = world.robot(0)

#If you'd like to show the print output, comment out this line
#clear_output()
display(kvis)
controls = widgets.HBox([])
display(controls)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

S2xhbXB0V2lkZ2V0KGhlaWdodD00MDAsIHNjZW5lPXt1J29iamVjdCc6IHt1J21hdHJpeCc6IFsxLCAwLCAwLCAwLCAwLCAwLCAtMSwgMCwgMCwgMSwgMCwgMCwgMCwgMCwgMCwgMV0sIHUndXXigKY=


HBox()

In [5]:
EPSILON = 1e-7

def ik_2R_analytical(L1,L2,x,y):
    """Solve IK for a 2R robot with link lengths L1 and L2.
    
    Returns a pair (N,solutions)
    - N: number of solutions (can be 0, 1, 2, or infinity)
    - solutions: a tuple of solution configurations. If N=infinity,
      solutions is the tuple (None,q2).
    """
    r2 = x**2+y**2
    c2 = (r2-L1**2-L2**2)/(2*L1*L2)
    if c2 > 1.0 or c2 < -1.0:
        return (0,())
    q2_1 = math.acos(c2)
    q2s = [q2_1]
    if abs(q2_1) > EPSILON and abs(q2_1-math.pi) > EPSILON:
        q2s.append(-q2_1)
    solns = []
    for q2 in q2s:
        ex = 1.0+math.cos(q2)
        ey = math.sin(q2)
        if abs(ex)<EPSILON and abs(ey) < EPSILON:
            #infinite solutions
            return (float('inf'),(None,q2))
        theta = math.atan2(y,x)
        etheta = math.atan2(ey,ex)
        solns.append((theta-etheta,q2))
    return (len(solns),solns)


def lab2a(L1,L2,L3,point):
    """
    TODO: complete me
    
    Compute all IK solutions of a 3R manipulator.  The joint axes in the reference configuration
    are ZYY, with each link's origin displaced by a given amount on the X axis.
    In:
    - L1, L2, L3: the link lengths
    - point: the target position (x,y,z)
    Out:
    - A pair (n,solutionList) where n is the number of solutions (either 0-4,
      or float('inf')) and solutionList is a list of all solutions.
      In the n=0 case, it should be an empty list [], and in the n=inf case
      it should give one example solution.
      Each solution is a 3-tuple giving the joint angles, in radians.
    """
    x,y,z = point
    
    #this returns no solution
    return (0,[])
    #this line would return one (incorrect) solution (0,0,0)
    return (1,[(0,0,0)])
    #this line would return two (incorrect) solutions
    return (2,[(0,0,0),(0,0,0)])
    #this line would return three (incorrect) solutions
    return (3,[(0,0,0),(0,0,0),(0,0,0)])
    #this line would return four (incorrect) solutions
    return (4,[(0,0,0),(0,0,0),(0,0,0),(0,0,0)])
    #this line would return infinite solutions, and one example (incorrect) solution (0,0,0)
    return (float('inf'),[(0,0,0)])

def fuzzy_eq(a,b,eps=1e-8):
    """Returns true if a is within +/- eps of b."""
    return abs(a-b)<=eps

def selfTest():
    res = lab2a(0,1,1,(2,0,0))
    print "Solutions for L1=0,L2=L3=1, target (2,0,0):"
    print res[0],"configurations"
    for q in res[1]:
        print "  ",q
    print "Should have two solutions, q1=q2=q3=0 and q1=q2=pi and q3=0"
    
    res = lab2a(1,1,1,(3,0,0))
    print "Solutions for L1=L2=L3=1, target (3,0,0):"
    print res[0],"configurations"
    for q in res[1]:
        print "  ",q
    print "Should have 1 solution, with q1=q2=q3=0"
    
    res = lab2a(0.25,1,1,(0,0,1))
    print "Solutions for L1=0.25, L2=L3=1, target (0,0,1):"
    print res[0],"configurations"
    for q in res[1]:
        print "  ",q
    print "Should have infinite solutions"
    
    res = lab2a(0,1,1,(0,0,1))
    print "Solutions for L1=0, L2=L3=1, target (0,0,1):"
    print res[0],"configurations"
    for q in res[1]:
        print "  ",q
    print "Should have infinite solutions"

In [6]:
#run this cell to bind your code to the visualization playback window
target = [0.,0.,0.]

ghosts = []
ghosts.append(kvis.add_ghost("solution2"))
ghosts.append(kvis.add_ghost("solution3"))
ghosts.append(kvis.add_ghost("solution4"))

def point_changed_cb(value):
    global target
    #solve
    solutions=lab2a(0.25,1,1,target)

    #update visualization of solutions
    if solutions[0] == 0 or solutions[0] == float('inf'):
        #no solutions, draw robot in transparent red
        for i in range(robot.numLinks()):
            kvis.set_color(robot.link(i),1,0,0,0.25)
        #hide ghosts
        for ghost in ghosts:
            kvis.set_color(ghost,0,0,0,0)
    else:
        #show/hide ghosts
        for i,ghost in enumerate(ghosts):
            if i+1 >= solutions[0]:
                print "Hiding ghost",i
                kvis.set_color(ghost,0,0,0,0)
            else:
                print "Setting ghost",i,"solution",solutions[1][i+1]
                kvis.set_color(ghost,1,1.0/(i+2),1)
                kvis.set_ghost_config(solutions[1][i+1],"solution"+str(i+2))
        for i in range(robot.numLinks()):
            kvis.set_color(robot.link(i),1,1,1)
        robot.setConfig(solutions[1][0])
    kvis.update()

point_widget = ipyklampt.EditPoint(target,[-2.5]*3,[2.5]*3,klampt_widget=kvis,callback=point_changed_cb)

controls.children = (point_widget,)

In [7]:
#testing... run this cell to verify your code
selfTest()

Solutions for L1=0,L2=L3=1, target (2,0,0):
0 configurations
Should have two solutions, q1=q2=q3=0 and q1=q2=pi and q3=0
Solutions for L1=L2=L3=1, target (3,0,0):
0 configurations
Should have 1 solution, with q1=q2=q3=0
Solutions for L1=0.25, L2=L3=1, target (0,0,1):
0 configurations
Should have infinite solutions
Solutions for L1=0, L2=L3=1, target (0,0,1):
0 configurations
Should have infinite solutions
